In [1]:
import pandas as pd
import numpy as np
import sys
import re
import os


df_ordered = pd.read_csv("../code/ordered.csv")
df_ordered['train'] = 1
df_orderedtest = pd.read_csv("../code/ordered_test.csv")
df_orderedtest['train']=0
df_orderedtest['units']=0

df_main = pd.concat([df_ordered,df_orderedtest])
df_main = df_main.drop(['Unnamed: 0','station_nbr','tmax','tmin','depart','dewpoint','wetbulb','heat','cool','sunrise','sunset','codesum','snowfall','stnpressure','sealevel','resultspeed'],axis=1)

df_ordered = []
df_orderedtest = []

In [2]:
df_main = df_main.sort_values(['store_nbr','item_nbr','date']).reset_index(drop=True)

df_main['Before_Event'] = (pd.rolling_mean(df_main['Condition'], window=3).shift(-3) > 0)
df_main['After_Event'] = (pd.rolling_mean(df_main['Condition'], window=3).shift(1) > 0)

df_main['Before_Sales'] = pd.rolling_mean(df_main['units'], window=6).shift(-6)
df_main['After_Sales'] = pd.rolling_mean(df_main['units'], window=6).shift(1)

print df_main.columns
print df_main.shape

item_nbr_list = list(set(df_main['item_nbr']))
store_nbr_list = list(set(df_main['store_nbr']))

/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  app.launch_new_instance()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=6,center=False).mean()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=6,center=False).mean()


Index([u'Condition', u'RA', u'SN', u'WEvent', u'avgspeed', u'date',
       u'item_nbr', u'preciptotal', u'resultdir', u'store_nbr', u'tavg',
       u'train', u'units', u'Before_Event', u'After_Event', u'Before_Sales',
       u'After_Sales'],
      dtype='object')
(5144517, 17)


In [3]:
from collections import defaultdict

zero_dict = defaultdict(list)

for store in store_nbr_list:
    print store
    for item in item_nbr_list:
        mask = ((df_main['item_nbr'] == item) & (df_main['store_nbr'] == store))
        if df_main.loc[mask,'units'].max() != 0:
            zero_dict[store].append(item)
        else:
            df_main.loc[mask] = np.nan

df_main = df_main.dropna()
print zero_dict
print df_main.columns
print df_main.shape

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
defaultdict(<type 'list'>, {1: [9, 28, 40, 47, 51, 89, 93, 99], 2: [5, 11, 44, 85, 93], 3: [2, 5, 45, 93, 102, 109], 4: [9, 27, 90, 93, 96, 97, 101], 5: [16, 20, 32, 37, 93, 105], 6: [9, 14, 45, 107], 7: [5, 7, 23, 93, 95], 8: [5, 23, 59, 84, 93, 100], 9: [5, 10, 42, 45, 93, 105], 10: [5, 21, 37, 86, 93], 11: [9, 61, 68, 86, 87, 110], 12: [16, 37, 79, 86, 93], 13: [5, 36, 46, 93, 104], 14: [9, 15, 25, 33, 50, 70, 88, 93], 15: [5, 15, 17, 45, 49, 61, 86, 93], 16: [16, 25, 39, 50, 64, 77, 85, 93], 17: [9, 37, 48, 71], 18: [5, 37, 57, 72], 19: [15, 16, 39, 69, 75, 83, 88, 93], 20: [5, 18, 41, 74, 93, 106], 21: [5, 44, 91, 109], 22: [9, 54, 68, 78, 93, 104], 23: [5, 29, 45, 56], 24: [6, 43, 60, 76, 97], 25: [9, 12, 45, 50, 81, 84, 93], 26: [2, 5, 45], 27: [9, 21, 68, 80, 93], 28: [2, 5, 35, 45], 29: [8, 41, 45, 56], 30: [13, 16, 44, 49, 51, 65], 31: [9, 45, 67, 93, 

In [4]:
df_main.to_csv('../code/non-zeros.csv',sep=',')

In [1]:
import pandas as pd
import numpy as np

df_main = pd.read_csv('../code/non-zeros.csv',sep=',')

df_main['year'] = pd.to_datetime(df_main['date'], infer_datetime_format=True).dt.year
df_main['month'] = pd.to_datetime(df_main['date'], infer_datetime_format=True).dt.month
df_main['weekday'] = pd.to_datetime(df_main['date'], infer_datetime_format=True).dt.weekday

df_main['units'] = df_main['units'].apply(lambda x: np.log(x+1))
df_main2 = pd.get_dummies(df_main, columns=['year','month','weekday','item_nbr','store_nbr'])

In [2]:
print df_main.columns

Index([u'Unnamed: 0', u'Condition', u'RA', u'SN', u'WEvent', u'avgspeed',
       u'date', u'item_nbr', u'preciptotal', u'resultdir', u'store_nbr',
       u'tavg', u'train', u'units', u'Before_Event', u'After_Event',
       u'Before_Sales', u'After_Sales', u'year', u'month', u'weekday'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

df_main2 = df_main2.drop(['Unnamed: 0','resultdir','tavg','preciptotal','date'],axis=1)

mask = (df_main2['train'] == 1)
mask_test = (df_main2['train'] == 0)
y_train = df_main2.loc[mask,'units']
X_train = df_main2.loc[mask,df_main2.columns != 'units']

X_test = df_main2.loc[mask_test,df_main2.columns != 'units']

scaling = MinMaxScaler(feature_range=(0,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

svr = SVR()
svr.fit(X_train, y_train)
df_main.loc[mask_test,'units'] = svr.predict(X_test)

In [ ]:
df_main = df_main.loc[mask_test]

df_main['units'] = df_main['units'].apply(lambda x: np.exp(x)-1)

df_orderedtest = pd.read_csv("../code/ordered_test.csv")
df_orderedtest['train']=0
df_orderedtest['units']=0

print df_orderedtest.shape

result = pd.merge(df_orderedtest, df_main, how='left', on=['date','item_nbr','store_nbr'])

print result.shape
print result.columns
result['id'] = result['store_nbr'].astype(str) + '_' + result['item_nbr'].astype(str) + '_' + result['date'].astype(str)
result = result.set_index('id')

result = result['units']
result = result.reset_index()
mask = (result['units'] < 0)
result.loc[mask,'units'] = 0
result.to_csv("../code/result_all_svm.csv",sep=",",index=False)